In [ ]:
!pip install jiwer==3.0.1

!pip install datasets==2.10.1

In [2]:
import os
import pandas as pd
import shutil
from datasets import load_dataset
from scipy.io import wavfile

In [ ]:
df_filename = []
df_path_to_wav = []
df_text = []
df_len_of_wav = []

# датасет разеделен на 16 частей 0-f
# нужно задать через запятую те части датасета, которые сейчас используются
nums_of_used_datasets = ['0', '1', '2', '3', '4', '5', '6', '7']

folders = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

for num in nums_of_used_datasets:
    print(num)
    for folder in folders:
        print(folder, end=' ')
        print()
        directory = f"/kaggle/input/sova-audiobooks-ru-{num}/{num}/{folder}"
        for filename in os.listdir(directory):
             if filename.endswith(".txt"):
                    txt_filename = filename
                    wav_filename = f"{filename.split('.')[0]}.wav"

                    txt_path = os.path.join(directory, txt_filename)
                    wav_path = os.path.join(directory, wav_filename)

                    df_filename.append(f'train/{wav_filename}')
                    df_path_to_wav.append(wav_path)
                    with open(txt_path,'r') as f:
                        df_text.append(f.read())
                    sr, wav = wavfile.read(wav_path)
                    df_len_of_wav.append(round(len(wav)/sr, 1 ))


df_sova_audiobooks = pd.DataFrame({
    'file_name': df_filename,
    'full_path': df_path_to_wav,
    'answer': df_text,
    'duration': df_len_of_wav,
})
df_sova_audiobooks['source'] = 'sova_audiobooks'
df_sova_audiobooks

In [4]:
df_sova_audiobooks['duration'].sum()/3600

136.84550000000002

In [5]:
os.mkdir("/kaggle/working/sova")
os.mkdir("/kaggle/working/sova/train")

In [ ]:
for index, row in df_sova_audiobooks.iterrows():
    if index % 10000 == 0:
        print(index)
    try:
        shutil.copy(row['full_path'], f"/kaggle/working/sova/{row['file_name']}")
    except:
        print('except', index)
        df_sova_audiobooks.drop(index, inplace=True)

In [ ]:
df_sova_audiobooks.drop(columns=['full_path'], inplace=True)

In [9]:
df_sova_audiobooks.to_csv('/kaggle/working/sova/metadata.csv')

In [ ]:
dataset = load_dataset("audiofolder", data_dir='/kaggle/working/sova')

In [ ]:
from huggingface_hub import login
login(token='')
dataset.push_to_hub(f"aanosov/mfdp_sova_part1", private=True)